# Tinyflux: Example with fictious temperature measurements

## Import Python Libraries und Settings

In [85]:
# Libraries
import pandas as pd
import csv
from tinyflux import TinyFlux, Point, FieldQuery, TagQuery, TimeQuery
from datetime import datetime, timezone, timedelta #querys welche gebraucht werden

# Settings
import warnings
warnings.filterwarnings("ignore")

# Abkürzungen für Query-Typen. Für die Übersichtlichkeit -> Ebenfalls in der Prüfung übernehmen
time = TimeQuery()
tags = TagQuery()
field = FieldQuery()

## Load function for creating a dataframe from query results into memory

In [86]:
#Kann 1:1 übernommen werden, einfach attribute anpassen wie time, sensor_id etc.

# Define the function to transform the list of Point objects into a DataFrame
def points_to_dataframe(points_list):
    data = []
    
    # Extract the relevant information
    for point in points_list:
        data.append({
            "time": point.time,  # Access time attribute
            "sensor_id": point.tags['sensor_id'],  # Access sender_id from tags dictionary
            "temperature": point.fields['temperature'],  # Access receiver_id from tags dictionary
            "status": point.tags['status'],# Access status from tags dictionary
        })

    # Create a DataFrame
    df = pd.DataFrame(data)

    # Convert time to datetime
    df['time'] = pd.to_datetime(df['time'])

    return df

## Initialize the TinyFlux database
## (Leere temperature.db wird generiert)

In [87]:
# Erstellung der Tinyflux-Datenbank (wird bereitgestellt)
db = TinyFlux("temperatures.db")

## Importing a CSV file and show first descriptive statistics

In [88]:
# CSV-Datei in Dataframe laden
df = pd.read_csv('example_data/synthetic_temperature_readings.csv')

# Das Format des Zeitstempels in Pandas setzen
df['time']= pd.to_datetime(df['time'], format='mixed')

In [ ]:
# Erste 10 Records des Dataframes anzeigen
df.head(10)

In [ ]:
df.info()

In [ ]:
# Deskriptive Statistiken des Dataframes
df.describe(include='all')

## Writing a CSV file into Tinyflux / Task1 
## (temperatures.db wird mit Daten gefüllt)


In [89]:
# Read CSV and insert data into TinyFlux
with open("example_data/synthetic_temperature_readings.csv", mode="r") as file:
    reader = csv.DictReader(file)
    for row in reader:
        point = Point(
            time=datetime.strptime(row["time"], "%Y-%m-%d %H:%M:%S"), #here is time-granularity defined, has to be as in csv file
            # example daily granularity: time = datetime.strptime(row["time"], "%Y-%m-%d")
            measurement="blockchain_transactions", #tabellenname
            fields={
                "temperature": float(row["temperature"])
                },  # Only numeric fields
            tags={
                "sensor_id": row["sensor_id"],
                "status": row["status"],  # Move non-numeric fields to tags
            }
        )
        db.insert(point)

## Writing manually into Tinyflux / Task2
## (in temperatures.db "default" data)

In [ ]:
# Create manual records
p1 = Point(
    time=datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc), #Granularitätsstufe Jahr, Monat, Tag, Stunden
    tags={"sensor_id": "20", "status": "operational"},
    fields={"temperature": 15.240971}
)

p2 = Point(
    time=datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc),
    tags={"sensor_id": "30", "status": "maintenance"},
    fields={"temperature": 20.517200}
)

# Insert into the DB.
db.insert_multiple([p1, p2])

## Querying by Time / Task3
## Greater than or equal to

In [ ]:
# Definieren der Query
time_query_conditions = (time >= datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc)) # greater than or equal to January 1, 2024, at midnight (in UTC).

# Datenabfrage mittels der definierten Query
my_results = db.search(time_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_time = points_to_dataframe(my_results)
df_result_time

## Querying by time / Task3
## Between two specific Dates

In [ ]:
# Query Records Between Two Specific Dates/Times
start_time = datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc)
end_time = datetime(2024, 12, 31, 23, 59, 59, tzinfo=timezone.utc)

# Query records between two dates
time_query_conditions = (start_time <= time <= end_time)

# Datenabfrage mittels der definierten Query
my_results = db.search(time_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_time = points_to_dataframe(my_results)
df_result_time

## Querying by time / Task3
## For a specific day

In [ ]:
# Define query conditions for a specific day (e.g., September 15, 2024)
start_time = datetime(2024, 9, 15, 0, 0, 0, tzinfo=timezone.utc)
end_time = datetime(2024, 9, 15, 23, 59, 59, tzinfo=timezone.utc)

# Query records between two dates
time_query_conditions = (start_time <= time <= end_time)

# Datenabfrage mittels der definierten Query
my_results = db.search(time_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_time = points_to_dataframe(my_results)
df_result_time

## Querying by time / Task3
## For a specific time

In [ ]:
# Definieren der Query
time_query_conditions = (time == datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc)) # greater than or equal to January 1, 2024, at midnight (in UTC).

# Datenabfrage mittels der definierten Query
my_results = db.search(time_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_time = points_to_dataframe(my_results)
df_result_time

In [ ]:
df_result_time.describe(include='all')

## Querying by Tag / Task5
## Specific Tag

In [ ]:
df.status.unique() # gibt alle Werte für Tag "status" an

In [70]:
# Definieren der Queries
tag_query_conditions_operational = (tags.status == "operational") # abfrage nach bestimmten tags
tag_query_conditions_maintenance = (tags.status == "maintenance")
tag_query_conditions_offline = (tags.status == "offline")

# Datenabfrage mittels der definierten Queries
my_results_operational = db.search(tag_query_conditions_operational)
my_results_maintenance = db.search(tag_query_conditions_maintenance)
my_results_offline = db.search(tag_query_conditions_offline)

# Erstellung eines Dataframes aus den Query-Resultaten
df_result_tag_operational = points_to_dataframe(my_results_operational)
df_result_tag_maintenance = points_to_dataframe(my_results_maintenance)
df_result_tag_offline = points_to_dataframe(my_results_offline)

## How?

In [ ]:
temp_by_location = df.groupby(['status'])

my_results_operational = db.search(temp_by_location)

df_result_tag_operational = points_to_dataframe(my_results_operational)


### Dataframe with operational status

In [ ]:
df_result_tag_operational.head(10) # Anzahl Abfragen aus dem Resultat "df_result_tag_operational"

In [ ]:
df_result_tag_operational.describe(include='all')

### Dataframe with maintenance status

In [ ]:
df_result_tag_maintenance.head(2)

In [ ]:
df_result_tag_maintenance.describe(include='all')

### Dataframe with offline status

In [ ]:
df_result_tag_offline.head(2)

In [ ]:
df_result_tag_maintenance.describe(include='all')

In [ ]:
# Unschöne Variante
tag_query = TagQuery()
q1 = db.search(tag_query.status == "operational")

for x in q1: 
    print(x)


## Querying by Field / Task4

In [ ]:
# Definieren der Queries
field_query_conditions = (field.temperature > 0)

# Datenabfrage mittels der definierten Queries
my_results = db.search(field_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_field = points_to_dataframe(my_results)

print("\n Dataframe with temperature measurements > 0:")
df_result_field.head(2)

In [ ]:
df_result_field.describe(include='all')

## Querying by Field / Task4
## total or average temperature values

In [ ]:
# Definieren der Queries / temperature grösser als 0 Grad
field_query_conditions = (field.temperature > 0)

# Datenabfrage mittels der definierten Queries
my_results = db.search(field_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
df_result_field = points_to_dataframe(my_results)

# Calculate total and average temperature
total_temperature = df_result_field['temperature'].sum()
average_temperature = df_result_field['temperature'].mean()

# Print the results
# print("\nDataframe with temperature measurements > 0:")
# print(df_result_field.head(2))  # Display the first 2 records
print(f"\nTotal temperature: {total_temperature}")
print(f"Average temperature: {average_temperature}")

#-------------------------------------------------------------------------------------

# Definieren der Queries / measured before Mach 31 2023
time_query_conditions = (time < datetime(2023, 3, 31, 0, 0, 0, tzinfo=timezone.utc))

# Datenabfrage mittels der definierten Queries
my_results = db.search(time_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
df_result_field = points_to_dataframe(my_results)

# Calculate total and average temperature
total_temperature = df_result_field['temperature'].sum()
average_temperature = df_result_field['temperature'].mean()

# Print the results
# print("\nDataframe with temperature measurements > 0:")
# print(df_result_field.head(2))  # Display the first 2 records
print(f"\nTotal temperature: {total_temperature}")
print(f"Average temperature: {average_temperature}")

#-------------------------------------------------------------------------------------

# Give average temperature of each status-value
temp_by_location = df.groupby(['status'])['temperature'].mean()

print(temp_by_location)

## Variations of combining queries

### Querying by Time and Tag

In [ ]:
# Querying all temperatures, that were measured before Mach 31 2023 with the status "operational"

# Definieren der Queries
time_query_conditions = (time < datetime(2023, 3, 31, 0, 0, 0, tzinfo=timezone.utc))
tag_query_conditions = (tags.status == "operational")

# Datenabfrage mittels der definierten Queries
my_results = db.search(time_query_conditions & tag_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_time_tag = points_to_dataframe(my_results)
df_result_time_tag.head(2)

In [ ]:
df_result_time_tag.describe(include='all')

### Querying by Tag and Field

In [ ]:
# Querying all temperatures higher than 20 degrees and with the status "maintenance"

# Definieren der Queries
tag_query_conditions = (tags.status == "maintenance")
field_query_conditions = (field.temperature > 20.0)

# Datenabfrage mittels der definierten Queries
my_results = db.search(tag_query_conditions & field_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_tag_field = points_to_dataframe(my_results)
df_result_tag_field.head(2)

In [ ]:
df_result_tag_field.describe(include='all')

## Variations of queries with multiple AND and OR conditions

In [ ]:
# Querying all measurements during winter months with temperatures below 0
# OR conditions are separated with "|" (not to be confused with "/", see example below)

# Definieren der Queries
time_query_conditions = (time <= datetime(2023, 2, 28, 0, 0, 0, tzinfo=timezone.utc)) | (time >= datetime(2023, 12, 1, tzinfo=timezone.utc)) # "or" mit |
field_query_conditions = (field.temperature < 0.0)

# Datenabfrage mittels der definierten Queries
my_results = db.search(time_query_conditions & field_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_time_field = points_to_dataframe(my_results)
df_result_time_field.head(2)

In [ ]:
df_result_time_field.describe(include='all')

In [ ]:
# Querying all measurements during summer months with temperatures above 0
# AND conditions are separated with "&"

# Definieren der Queries
time_query_conditions = (time <= datetime(2023, 8, 31, 0, 0, 0, tzinfo=timezone.utc)) & (time >= datetime(2023, 6, 1, tzinfo=timezone.utc))
field_query_conditions = (field.temperature > 0.0)

# Datenabfrage mittels der definierten Queries
my_results = db.search(time_query_conditions & field_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_time_field_2 = points_to_dataframe(my_results)
df_result_time_field_2.head(2)

In [ ]:
df_result_time_field_2.describe(include='all')

## How Tags Are Useful for Organizing and Filtering Data in a Time-Series Database:

### Tags in a time-series database (like TinyFlux, InfluxDB, etc.) are key-value pairs that help organize and filter data more efficiently. Here's why they are important:

### - Efficient Filtering: Tags allow for fast querying and filtering of records. For example, filtering all data where location="Zurich" allows you to quickly retrieve data points related to that specific location without needing to search through the entire dataset.

### - Group and Aggregate Data: Tags can be used to group and aggregate data, as shown above. You can group records by any tag (e.g., location, device_type, status), which is useful for reporting and trend analysis. For example, analyzing the average temperature for each location or counting the number of sensor devices.

### - Improved Data Organization: In time-series databases, tags categorize data meaningfully. A sensor_id or location tag gives context to each data point, making it easier to work with complex datasets.

### - Optimized Storage: Tags are indexed efficiently, allowing for fast lookups. Unlike fields (which store raw data), tags are meant for frequent queries and indexing, making the database more efficient when handling large datasets.

### In summary, tags are crucial in a time-series database as they enable faster searches, data organization, and aggregation, making it easier to extract insights from large sets of time-series data.